# Ejercicio: Crear un DataFrame Completo con SWAPI

## Objetivo:
El objetivo de este ejercicio es que los alumnos practiquen la obtención de datos de una API REST y su posterior procesamiento para crear un DataFrame en Pandas que contenga información detallada sobre personajes de Star Wars.

## Instrucciones:

1. Obtener los datos:

- Utilizar la API de Star Wars (SWAPI) para obtener datos sobre los personajes (people) de la saga.
- La URL base para obtener la lista de personajes es: https://swapi.dev/api/people/.
- Dado que los datos están paginados, deberán recorrer todas las páginas para obtener la lista completa de personajes.

2. Crear el DataFrame:

    Crear un DataFrame que incluya las siguientes columnas con la información detallada de cada personaje:

    <!-- - id: Identificador único del personaje.
    - name: Nombre del personaje.
    - height: Altura del personaje (en centímetros).
    - mass: Peso del personaje (en kilogramos).
    - hair_color: Color de cabello.
    - skin_color: Color de piel.
    - eye_color: Color de ojos.
    - birth_year: Año de nacimiento.
    - gender: Género.
    - homeworld: Nombre del planeta de origen (obtenido mediante una petición adicional a la API).
    - species: Nombre de la especie del personaje (obtenido mediante una petición adicional si el campo species no está vacío).
    - films: Lista de nombres de películas en las que aparece el personaje (obtenido mediante peticiones adicionales). -->

3. Procesamiento de datos:

    - Cada entrada en el campo homeworld de los personajes es un enlace a la API, desde el cual se debe obtener el nombre del planeta.

    - El campo species contiene una lista de enlaces. Si no está vacío, se debe obtener el nombre de la especie desde la API.

    - El campo films contiene una lista de enlaces a las películas. Se debe realizar una petición por cada enlace para obtener el título de la película.

4. Entrega del ejercicio:

    - Se debe entregar un archivo .ipynb (Jupyter Notebook) con el código que obtenga los datos, procese las respuestas y construya el DataFrame completo.

    - También deben incluir en el notebook una breve descripción de cómo organizaron el proceso para obtener los datos y construir el DataFrame.

Tips:

- Utilicen la librería requests para realizar las peticiones HTTP.
- Tengan en cuenta que la API utiliza paginación, por lo que deberán iterar sobre las páginas hasta obtener todos los datos.
- Los datos relacionados (homeworld, species, films) requieren realizar peticiones adicionales, así que gestionen adecuadamente los tiempos de espera (pueden considerar utilizar la función time.sleep() para evitar sobrecargar la API).
- Si encuentran campos con datos faltantes o errores, documenten cómo lo manejan en su notebook.

In [2]:
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed


In [ ]:
url_planeta = 'https://swapi.dev/api/planets/'
url_peliculas = 'https://swapi.dev/api/films/'
for i in range(1,84):
    url_people = f'https://swapi.dev/api/people/{i}'
res = requests.get(url_people)
swar = res.json()
swar

{'count': 82,
 'next': 'https://swapi.dev/api/people/?page=2',
 'previous': None,
 'results': [{'name': 'Luke Skywalker',
   'height': '172',
   'mass': '77',
   'hair_color': 'blond',
   'skin_color': 'fair',
   'eye_color': 'blue',
   'birth_year': '19BBY',
   'gender': 'male',
   'homeworld': 'https://swapi.dev/api/planets/1/',
   'films': ['https://swapi.dev/api/films/1/',
    'https://swapi.dev/api/films/2/',
    'https://swapi.dev/api/films/3/',
    'https://swapi.dev/api/films/6/'],
   'species': [],
   'vehicles': ['https://swapi.dev/api/vehicles/14/',
    'https://swapi.dev/api/vehicles/30/'],
   'starships': ['https://swapi.dev/api/starships/12/',
    'https://swapi.dev/api/starships/22/'],
   'created': '2014-12-09T13:50:51.644000Z',
   'edited': '2014-12-20T21:17:56.891000Z',
   'url': 'https://swapi.dev/api/people/1/'},
  {'name': 'C-3PO',
   'height': '167',
   'mass': '75',
   'hair_color': 'n/a',
   'skin_color': 'gold',
   'eye_color': 'yellow',
   'birth_year': '112BB

In [18]:
def find_name(url):
    try:
        res = requests.get(url)
        data = res.json()
        return data['name']
    except:
        return 'Error'
    
def find_film(films):
    lista_titles = []
    for f in films:
        try:
            res = requests.get(f)
            data = res.json()
            lista_titles.append(data['title'])  
        except:
            'Error'
            continue
    return lista_titles
    

In [17]:
swar_list = [{'id':1,
              'name':swapi['name'],
              'height':swapi['height'],
              'mass':swapi['mass'],
              'hair_color':swapi['hair_color'],
              'skin_color':swapi['skin_color'],
              'eye_color':swapi['eye_color'],
              'birth_year':swapi['birth_year'],
              'gender':swapi['gender'],
              'homeworld':find_name(swapi['homeworld']),
              'species':find_name(swapi['species'][0]) if swapi['species'] else 'No tiene specie',
              'films':find_film(swapi['films'])}for swapi in swar['results']]
df_swar = pd.DataFrame(swar_list)
df_swar

,id,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,species,films
0,1,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,Tatooine,No tiene specie,"[A New Hope, The Empire Strikes Back, Return o..."
1,1,C-3PO,167,75,n/a,gold,yellow,112BBY,n/a,Tatooine,Droid,"[A New Hope, The Empire Strikes Back, Return o..."
2,1,R2-D2,96,32,n/a,"white, blue",red,33BBY,n/a,Naboo,Droid,"[A New Hope, The Empire Strikes Back, Return o..."
3,1,Darth Vader,202,136,none,white,yellow,41.9BBY,male,Tatooine,No tiene specie,"[A New Hope, The Empire Strikes Back, Return o..."
4,1,Leia Organa,150,49,brown,light,brown,19BBY,female,Alderaan,No tiene specie,"[A New Hope, The Empire Strikes Back, Return o..."
5,1,Owen Lars,178,120,"brown, grey",light,blue,52BBY,male,Tatooine,No tiene specie,"[A New Hope, Attack of the Clones, Revenge of ..."
6,1,Beru Whitesun lars,165,75,brown,light,blue,47BBY,female,Tatooine,No tiene specie,"[A New Hope, Attack of the Clones, Revenge of ..."
7,1,R5-D4,97,32,n/a,"white, red",red,unknown,n/a,Tatooine,Droid,[A New Hope]
8,1,Biggs Darklighter,183,84,black,light,brown,24BBY,male,Tatooine,No tiene specie,[A New Hope]
9,1,Obi-Wan Kenobi,182,77,"auburn, white",fair,blue-gray,57BBY,male,Stewjon,No tiene specie,"[A New Hope, The Empire Strikes Back, Return o..."
